[![View On GitHub](https://img.shields.io/badge/View_in_Github-grey?logo=github)](https://gitlab.com/your_username/your_repository/-/blob/main/examples/showcase.ipynb)
![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)

# Cracking the Code: Practical Methods for Testing Differential Privacy

In sarus we build privacy safe analytics tools and one of our initiatives is an open source library called [qrlew](https://qrlew.readthedocs.io/en/latest/). Qrlew is used to turn sql queries into privacy save ones under the framework of [Differential Privacy (DP)](https://en.wikipedia.org/wiki/Differential_privacy). As the library was maturing and more features were supported we needed to test that the results generated from DP rewritten queries were actually coherent with the theory. Thanks to the approach presented here, we actually found a bug in Qrlew and corrected it straight away. This motivated us to open source our testing methodology and tools to the developers and researchers community so, let's dive in.

To build trustworthy data analysis systems, it’s important to verify that differential privacy mechanisms truly provide the privacy guarantees they promise. While there have been other attempts to create open-source libraries for differential privacy testing—such as Google’s Differential Privacy [Stochastic Tester](https://github.com/google/differential-privacy/blob/main/cc/testing/README.md), [DP Auditorium](https://github.com/google/differential-privacy/tree/main/python/dp_auditorium), and potentially others, our approach focuses on making it exceptionally easy to create datasets that resemble real-life scenarios and to effortlessly test them with any SQL query. In this ready-to-run Jupyter notebook on Google Colab, we introduce practical methods for testing differential privacy results. Adopting the perspective of an adversary attempting to breach privacy, we demonstrate how hypothesis testing, specifically using the Neyman–Pearson lemma, can assess whether a mechanism effectively protects individual entries in a dataset.

Here, we also introduce the dp_tester repository that implements these testing techniques, showcasing how straightforward it is to apply them to your own differential privacy mechanisms. Through hands-on code examples and interactive experiments, you’ll see how easy it is to empirically validate and verify the privacy properties of your implementations using our tools.

This practical approach bridges the gap between theoretical guarantees and real-world applications, empowering developers and researchers to ensure their differential privacy solutions are both robust and effective—all within a convenient and user-friendly Colab environment.

---

## Table of Contents

- [How to test DP results](#intro)
- [Experimental settings & Results](#dp_testing_in_practice)
  1) [Dataset Generation](#dataset_generation)
  2) [Collecting DP Results](#collecting_dp_results)
  3) [Partitioning Results](#paritioning_results)
  4) [Compute Empirical Epsilons](#compute_empirical_epsilons)
- [Conclusions](#conclusions)
- [References](#theoretical_backgroud)

<a id="intro"></a>
## How to test DP results 

Differential privacy (DP) offers a robust mathematical framework to ensure that the output of a mechanism $\mathcal{M}$ does not compromise the privacy of any individual in a dataset. Formally a mechanism $\mathcal{M}$ is ($\epsilon$,$\delta$)-differential private if:

$$ Pr \left[ \mathcal M\left( D_0 \right) \in S\right] \leq e^{\epsilon} Pr\left[ \mathcal M\left(D_1\right)\in S \right] + \delta $$

$\forall \; D_0$ and $D_1$, where $|D_0-D_1| \leq 1$ (neighboring datasets differing by at most one individual), $\forall \; S$ any possible output set. $\epsilon$ is the privacy loss parameter, and $\delta$ is a small failure probability.

### Adversary’s Perspective

To test whether a mechanism truly satisfies differential privacy, we can adopt the perspective of an adversary attempting to determine whether a specific user is included in a dataset. The adversary’s goal is to distinguish between outputs from $D_0$ (with the user) and $D_1$ (without the user).

### Hypothesis Testing Framework

This scenario can be framed as a hypothesis testing problem using the Neyman–Pearson lemma:
- Null Hypothesis ($H_0$): The output Y originates from $\mathcal{M}(D_0)$.
- Alternative Hypothesis ($H_1$): The output Y originates from $\mathcal{M}(D_1)$.

The Neyman–Pearson lemma is a fundamental result in statistical hypothesis testing that provides a method for constructing the most powerful test for distinguishing between two simple hypotheses and has applications across different domains such as in medicine, physics, economy etc. The lemma states that for a given significance level  $\alpha$  (the maximum allowable probability of a Type I error), the test for deciding between  $H_0$  and  $H_1$  is the one that rejects  $H_0$  in favor of  $H_1$  when the likelihood ratio $\Lambda(Y)$  exceeds a certain threshold $\tau$:
$$\Lambda(Y) = \frac{f_1(Y)}{f_0(Y)} \geq \tau$$ 
The threshold $\tau$ is chosen so that the test has the desired significance level $\alpha$:
$$ \alpha = Pr_{H_0}[\Lambda(Y) \geq \tau] $$
The lemma guarantees that among all possible tests with significance level  $\alpha$ , the likelihood ratio test is the most powerful—it has the highest probability of correctly rejecting  $H_0$  when  $H_1$  is true (i.e., the lowest Type II error  $\beta$ i.e. )

### Computing The Empirical Epsilon

In our scenario, in order to compute the likelihood ratio, a type I and type II error for our hypothesis test we can consider a sub region of the output space delimited by an arbitrary $S_{\tau}$, it implies that we can write the following from the formal definition above:
$$ Pr \left[ \mathcal M\left( D_0 \right) \in S_{\tau}\right] \leq e^{\epsilon} Pr\left[ \mathcal M\left(D_1\right)\in S_{\tau} \right] + \delta \quad \quad \forall \; D_0, D_1, |D_0-D_1| \leq 1, \quad \forall \; S_{\tau} $$
where $S_{\tau} \in \{\frac{Pr \left[ \mathcal M\left( D_0 \right) \in S\right]} {Pr\left[ \mathcal M\left(D_1\right)\in S \right]} \leq \tau \}$.

Now we can define the probability of false positive as $FP_{\tau}=1-Pr\left[ \mathcal M\left( D_0 \right) \in S_{\tau}\right]$ (type I error), when the null hypothesis is true but rejected, and the probability of False Negative (type II error) as $FN_{\tau}=Pr\left[ \mathcal M\left( D_1 \right) \in S_{\tau}\right]$. In the graph below there is a schematic representation of our FP and FN definitions ($P_0$ and $P_1$ are respectively $Pr\left[ \mathcal M\left( D_o \right) \in S_{\tau}\right]$ and $Pr\left[ \mathcal M\left( D_1 \right) \in S_{\tau}\right]$).

<div style="text-align: center;">
    <img src="figure_fp-fn_2.png" width="400" alt="" />
</div>

If we plug our type I and type II error in the original definition and we re-arrange a bit in order to isolate $\epsilon$ we get the following:
$$ \epsilon \geq ln\left(\frac {1 - \delta - FP_{\tau}} {FN_{\tau}}\right) \quad \quad \forall \; D_0, D_1, |D_0-D_1| \leq 1, \quad \forall \; S_{\tau}  $$
Since this relation is valid for all possible neighboring datasets It is true as well for a set neighboring datasets and moreover it is true as well for the neighboring dataset that maximizes the empirical epsilon. Thus, it is implied that:
$$ \epsilon \geq \epsilon^{*} = \max_{\tau, \pi} \left[ ln\left(\frac {1 - \delta - FP_{\tau}} {FN_{\tau}}\right) \right] \quad \quad \forall \; (D_0, D_1) \in \{\pi_1, \pi_2, ... \pi_n\}, \pi_i = (D^i_0, D^i_1), |D^i_0-D^i_1| \leq 1, \quad \forall \; S_{\tau} $$
This relation tells us that the $\epsilon^{*}$ should never overcome the actual $\epsilon$ used by the mechanism otherwise there are potential vulnerability in the mechanism implementation. Since $FP_{\tau}$ and $FN_{\tau}$ are unknown probabilities which we need to estimate.
$$ \epsilon \stackrel{?}{\geq} \hat{\epsilon^{*}} = \max_{\tau, \pi} \left[ ln\left(\frac {1 - \delta - \widehat{FP_{\tau}}} {\widehat{FN_{\tau}}}\right) \right] \quad \quad \forall \; D_0, D_1 \in \{\pi_1, \pi_2, ... \pi_n\}, \pi_i = (D^i_0-D^i_1), |D^i_0-D^i_1| \leq 1, \quad \forall \; S_{\tau}  $$
where the $?$ indicates that this formula may not hold if $\widehat{FP_{\tau}}$ and $\widehat{FN_{\tau}}$ are poor estimators. We use empirical probability estimators for $FP_{\tau}$ and $FN_{\tau}$, we count a the number of occurrences in a specific bucket over a sufficiency large number or experiments (on the order of 10k or more). In this way the analysis presented so far is conducted for each individual bucket thus the $\hat{\epsilon^{*}}$ is the maximum across all partitions along with all neighboring datasets.

### Interpreting the Results

The test fails if $\hat{\epsilon^{*}}$ is significantly greater that $\epsilon$. This indicates that the mechanism does not meet the privacy criteria, revealing a potential vulnerability that needs to be addressed. If $\hat{\epsilon^{*}}$ across all tested dataset pairs is less than or equal to the claimed $\epsilon$, the mechanism appears to uphold the differential privacy guarantee for those cases.
This procedure has some limitations, since we cannot test all possible neighboring datasets, passing the test does not conclusively prove that the mechanism is differentially private in all scenarios.

<a id="dp_testing_in_practice"></a>
## Experimental settings

In this section we go from theory to practice in 4 steps:
1) **Dataset Generation**: a brief description of the datasets caracteristics we generated for the experiments.
2) **Collecting DP Results**: description on how we collected our DP results.
3) **Partitioning Results**: description and tools for partitioning the output space into buckets needed for computing counts.
4) **Compute Empirical Epsilons**: here we compute empirical epsilons for all partitions and compare it to the actual epsilon used for the experiments.

<a id="dataset_generation"></a>
### Dataset Generation

In the following cells we create a postgres database and we push our $D_0$ and $D_1$ datasets which differ by the removal of the data relative to 1 user.

The dataset consists in 2 tables:
- `users`: it has 100 lines each related to a distinct user. The columns are `id: (int) UNIQUE` identifying the user and `income: (float)` is a random number drawn from $\mathcal{N}(\mu=40000, \sigma=10000)$
- `transactions`: it has 10000 entries, The columns are `id: (int) UNIQUE` the transaction id, `user_id: (int)` the user who is making the transaction (it is one of the user in users), `spent: (float)` is a random number drawn from $\mathcal{U}(a=5,b=500)$, `store_id: (int)` identifier of the store, there are 200 unique stores, `other_id: (int)` is the identifier of some arbitrary feature of the user, there are 10 possible unique other ids.


In the `transactions` table the user `0` has at most 500 contributions while the remaining transactions are split uniformly among the remaining users. Moreover, the user `0` likes to make transactions in all stores and he likes them differently: the higher the store_id the higher the frequency the user `0` likes to make transactions. In other wards user `0` affects all the stores in different way. The other users don't have any particular preference among the stores so the pick them randomly. Lastly, each user affects only 1 `other_id`.

$D_0$ is pushed to the default postgres schema while $D_1$ which has no information about the user with id=0 is pushed to the `D_1` schema. 

---

In [ ]:
%%capture
# Create a postgres database
# Inspired by https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/postgresql.ipynb#scrollTo=YUj0878jPyz7
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-14
# Start postgresql server
!sudo sed -i "s/port = 5432/port = 5433/g" /etc/postgresql/14/main/postgresql.conf
!sudo service postgresql start
# Set password
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pyqrlew-db'"

In [ ]:
from dp_tester.generate_datasets import generate_D_0_dataset, generate_adj_datasets
from dp_tester.constants import D_1

generate_D_0_dataset()
generate_adj_datasets(D_1, user_id=0)

<a id="collecting_dp_results"></a>
### Collecting DP Results

To collect DP results, we use the `dp_results_from_sql_query` function which uses `SqlAlchemyQueryExecutor` query executor to submit a query to the database, we use `PyqrlewDpRewriter` to compile the query with differential privacy with the provided privacy parameters
and the we use `PyqrlewTableRenamer` to change tables qualifying name to forward the query towards $D_1$.

The DP query is generated ones and is submitted many times (as much as indicated by `runs` parameter) to both $D_0$ and $D_1$.
Then results for each of the 2 dataset are stored in dictionary.

---

In [ ]:
from dp_tester.results_collector import dp_results_from_sql_query
from dp_tester.query_executors import SqlAlchemyQueryExecutor
from dp_tester.dp_rewriters import PyqrlewDpRewriter
from dp_tester.table_renamers import PyqrlewTableRenamer
from dp_tester.constants import D_0
import json

query = (
    "SELECT store_id, SUM(spent) AS spent_per_store FROM transactions GROUP BY store_id"
)
epsilon = 1.0
delta = 1e-4
runs = 5000

query_executor = SqlAlchemyQueryExecutor()
dp_rewriter = PyqrlewDpRewriter(engine=query_executor.engine)
table_renamer = PyqrlewTableRenamer(dp_rewriter.dataset)

results = dp_results_from_sql_query(
    non_dp_query=query,
    epsilon=epsilon,
    delta=delta,
    runs=runs,
    dp_rewriter=dp_rewriter,
    query_executor=query_executor,
    table_renamer=table_renamer,
    d_0=D_0,
    adjacent_ds=[D_1],
)

# save results
with open("results.json", "w") as outfile:
    json.dump(obj=results, fp=outfile)

<a id="paritioning_results"></a>
### Partitioning Results

Results structure can be very variate depending on the query and for a single query there are many ways to its results into buckets.
Here, we attempt to be generic and leave to the user the decision on how to partition the results into buckets. We have a generic function that
iterate over results and applies a user provided function which given the a single query results it returns a list of integers each associated
to the index of the corresponding bucket.

This approach should provide great flexibility while still remaining generic. 

For our case each query creates a list of row where each row is: `group_id, quantity`. For results of this structure we have implemented
such function as follow:

#### Buckets
Let's say that `quantity` is defined as $x \in \mathbb{R} \cup \{ \emptyset \} $ and `group_id` as $g\in G = \{ g_1, g_2, \dotsc, g_{M-1}\} \cup \{\emptyset\}$. We can discretize the space of $x$ in a set of $ I = \{ I_1, I_2, \dotsc, I_{N-1} \} \cup \{ \emptyset \}$ intervals where each interval $I_i$ is defined as: $I_i = [ a_{i},\ b_i ), \quad \text{for } i = 1, 2, \dotsc, N-1$, where N is the number of bins in which the space is discretized. The result space is therefore defined as  

$$ S = \{G \times I \} \cup \{ \emptyset \} $$ 

where the cross product $G \times I$ is the set paris $\{(g, I_i) | g\in G, I_i \in I\}$. So results are partitioned into $M \times N$ partitions or buckets. The method `generate_buckets` of the `QuantityOverGroups` object does exactly this. It uses the results to find quantities minimum and maximum in order then to generate NBINS intervals.

#### Counts

The function `results_to_bucket_ids` takes all the overall results and a used defined function, in this case provided by `QuantityOverGroups` object which associate an query results into bucket indexes as follow:
for each group id (store id in our case), we return the index of the corresponding bucket if the pair (group id, quantity) is present in the result other wise the index of the last bucket is returned (the one relative to the $\emptyset$). Notice that we take into account as well for groups that don't appear in the results. For each query results a list MxN bucket index is returned.

As a next step `counts_from_indexes` counts the indexes or in other wards, the number of occurrences of a particular result in a specific bucket.

---

In [ ]:
from dp_tester.generate_datasets import N_STORES
from dp_tester.partitioners import QuantityOverGroups
from dp_tester.analyzer import results_to_bucket_ids, counts_from_indexes
from dp_tester.typing import OverallResults
import typing as t


NBINS = 20

# read results
with open("results.json") as infile:
    results = t.cast(OverallResults, json.load(infile))

partitioner = QuantityOverGroups(groups=list(range(N_STORES)))
partitioner.generate_buckets(results, n_float_buckets=NBINS)
bucket_ids = results_to_bucket_ids(results, partitioner.bucket_id)

counts_d_0 = counts_from_indexes(bucket_ids[D_0], len(partitioner.buckets))
counts_d_1 = counts_from_indexes(bucket_ids[D_1], len(partitioner.buckets))

<a id="compute_empirical_epsilons"></a>
### Compute Empirical Epsilon

Now we that we have counts for each bucket we can estimate false positive and false negative errors and therefore compute the empirical epsilon.
Let's first compute the ratio between counts and sort them:

$$ \frac {c^{D_0}_{0}}{c^{D_1}_{0}} \geq \frac {c^{D_0}_{1}}{c^{D_1}_{1}} \geq  ... \geq \frac {c^{D_0}_{n-1}}{c^{D_1}_{n-1}}$$ 

where $n$ is the $M \times N$-th bucket and $\sum_i c^{D_0}_{i} = \sum_i c^{D_1}_{i} = C$ where C is the number or runs times the number of buckets. We can define a Neyman–Pearson lemma threshold as follow:

$$ \widehat{FP_{\tau_i}} = 1-\sum_{j\leq\tau_i}\frac{c^{D_0}_{j}}{C};\qquad \widehat{FP_{\tau_i}} = \sum_{j\leq\tau_i} \frac{c^{D_1}_{j}}{C} $$

Where C is total number of runs, $ \tau_i = \bigcup_{j>i} S_j $ in which S is sorted b where we vary the threshold across the results space. 

The empirical epsilon be computed as:

$$ \hat{\epsilon^{*}_T} = \max_{\tau_i, \pi} \left[ ln\left(\frac {1 - \delta - \widehat{FP_{\tau_i}}} {\widehat{FN_{\tau_i}}}\right), ln\left(\frac {1 - \delta - \widehat{FN_{\tau_i}}} {\widehat{FP_{\tau_i}}}\right) \right]$$

notice a couple of things:
1) by switching $D_0$ and $D_1$ we can include in our test all cases where a user has been added to the dataset. This has been already computed and is sufficient to include the second term in the max arguments where $\widehat{FP_{\tau_i}}$ and $\widehat{FN_{\tau_i}}$ are replaced.

2) The $T$ in $\hat{\epsilon^{*}_T}$ indicates that we are considering only buckets with at least $T$ counts: $\sum_{j\leq i}\frac{c^{D_0}_{j}}{C} \geq T$ and $\sum_{j\leq i}\frac{c^{D_1}_{j}}{C} \geq T$. We would like to avoid buckets with very few counts since we suppose that the error in estimating FPs and FNs is significantly high and it can falsify our test. One can either increase the number of runs with the hope that to populate a bit more such buckets or can calibrate $T$. $T$ can be calibrated by running an experiment such as counting the lines of the `users` table. The output is a response from a Laplace mechanism with sensibility of 1, this is a very simple mechanism for which $\hat{\epsilon^{*}_T}$ and $\epsilon$ are the closest. T would be the lowest possible for which the test passes.

---

In [ ]:
from dp_tester.analyzer import empirical_epsilon

COUNT_THRESHOLD = 5

empirical_eps = empirical_epsilon(
    counts_d_0, counts_d_1, delta=delta, counts_threshold=COUNT_THRESHOLD
)

print(f"Epsilon used during the experiment: {epsilon}")
print(f"Max empirical epsilon found: {empirical_eps}")
print(f"Did the test passed? {empirical_eps < epsilon}")

As you can see the test passes as we wanted to show.

<a id="conclusions"></a>
## Conclusions

In this notebook, we showed a methodology for testing any differentially private results using the principles of hypothesis testing and Neyman–Pearson lemma. We also introduced the `dp_testing` library with tools to easily experiment with realistic datasets creation, to obtain differentially private results from it and to analyze such results with hypothesis testing framework. Thanks to these tools we were able to find a problem with one of our DP mechanism implementation and to fix it.


<a id="theoretical_backgroud"></a>
## References

- [Differencial Privacy](https://maxkasy.github.io/home/files/other/ML_Econ_Oxford/differential_privacy.pdf)
- [Wilson et al.](https://arxiv.org/abs/1909.01917)
- [Kairouz et al.](https://arxiv.org/abs/1311.0776)
- [Milad Nasr et al.](https://arxiv.org/abs/2101.04535)